RFM Analysis is used to understand and segment customers based on their behaviour.
RFM stands for recency,frequency,and monetary value,which are three key metrics that provide information about customer engagement,loyalty,and value to a business.

Using RFM Analysis,a business can assess customers:
Recency:The date they made their last purchase

Frequency:How often they make purcahses

Monetary value: The amount spend on purchases


In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default="plotly_white"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv("/content/drive/MyDrive/rfm_data.csv")
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


The above dataset includes customerID,Purchase Date,Transaction Amount.

We can calculate RFM values for each customer and analyze their patterns and behaviours.

Calculating RMF Values

In [5]:
from datetime import datetime
# convert 'PurchaseDate' to datetime
data['PurchaseDate']=pd.to_datetime(data['PurchaseDate'])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 47.0+ KB


# Recency: subtracted the purchase date from the current date and extracted the number of days

In [7]:
# Calculate Recency
data['Recency']=(datetime.now().date()-data['PurchaseDate'].dt.date).dt.days

# Frequency: group the date by customerID and count the number of unique orderID which determines the number of purchase made by each customer


In [8]:
# Calculate Frequency
frequency_data=data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID':'Frequency'},inplace=True)
data=data.merge(frequency_data,on='CustomerID',how='left')


# Monetary Value: group the data by customerID and summed the TransactionAmount values to calculate the total amount spent by each customer

In [9]:
# Calculate Monetary Value
monetary_data=data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount':'MonetaryValue'},inplace=True)
data=data.merge(monetary_data, on='CustomerID',how='left')

In [10]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,302,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,302,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,302,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,302,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,302,1,739.56


# Calculating RMF Scores

In [11]:
# Define scroing crieria for each RMF Value
recency_scores=[5,4,3,2,1] # Higher score lower for recency(more recent)
frequency_scores=[1,2,3,4,5] # Higher score for higher frequency
monetary_scores=[1,2,3,4,5] # Higher score for higher monetary value

use pd.cut() function to divide recency,frequency,and monetary values into bins.

we definde 5 bins for each value and assign the corressponding scores to each bin.

In [12]:
# calculating RMF Scores
data['RecencyScore']=pd.cut(data['Recency'],bins=5,labels=recency_scores)
data['FrequencyScore']=pd.cut(data['Frequency'],bins=5,labels=frequency_scores)
data['MonetaryScore']=pd.cut(data['MonetaryValue'],bins=5,labels=monetary_scores)

In [13]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,302,1,943.31,1,1,2
1,2188,2023-04-11,463.70,Product A,176819,London,302,1,463.70,1,1,1
2,4608,2023-04-11,80.28,Product A,340062,New York,302,1,80.28,1,1,1
3,2559,2023-04-11,221.29,Product A,239145,London,302,1,221.29,1,1,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,302,1,739.56,1,1,2


In [ ]:
data.tail()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
 9   RecencyScore        1000 non-null   category      
 10  FrequencyScore      1000 non-null   category      
 11  MonetaryScore       1000 non-null   category      
dtypes: category(3), datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 81.7+ KB


In [15]:
# convert RFM scores to numeric type
data['RecencyScore']=data['RecencyScore'].astype(int)
data['FrequencyScore']=data['FrequencyScore'].astype(int)
data['MonetaryScore']=data['MonetaryScore'].astype(int)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
 9   RecencyScore        1000 non-null   int64         
 10  FrequencyScore      1000 non-null   int64         
 11  MonetaryScore       1000 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 101.6+ KB


# RFM Value Segmentation

# segmentation is done using the pd.qcut() function which evenly distributes scores between segments

In [17]:
# Calculate RFM score by combining the individual scores
data['RMF_Score']=data['RecencyScore']+data['FrequencyScore']+data['MonetaryScore']

# Create RFM segments based on the RFM Score
segment_labels=['Low-Value','Mid-Value','High-Value']
data['Value Segment']=pd.qcut(data['RMF_Score'],q=3,labels=segment_labels)

In [18]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RMF_Score,Value Segment
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,302,1,943.31,1,1,2,4,Low-Value
1,2188,2023-04-11,463.70,Product A,176819,London,302,1,463.70,1,1,1,3,Low-Value
2,4608,2023-04-11,80.28,Product A,340062,New York,302,1,80.28,1,1,1,3,Low-Value
3,2559,2023-04-11,221.29,Product A,239145,London,302,1,221.29,1,1,1,3,Low-Value
4,9482,2023-04-11,739.56,Product A,194545,Paris,302,1,739.56,1,1,2,4,Low-Value


In [19]:
data.tail()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RMF_Score,Value Segment
995,2970,2023-06-10,759.62,Product B,275284,London,242,1,759.62,5,1,2,8,High-Value
996,6669,2023-06-10,941.50,Product C,987025,New York,242,1,941.50,5,1,2,8,High-Value
997,8836,2023-06-10,545.36,Product C,512842,London,242,1,545.36,5,1,2,8,High-Value
998,1440,2023-06-10,729.94,Product B,559753,Paris,242,1,729.94,5,1,2,8,High-Value
999,4759,2023-06-10,804.28,Product D,467544,New York,242,1,804.28,5,1,2,8,High-Value


In [20]:
data.to_csv('RFM_DATA_NOW',index=False)

# Segment Distribution

In [21]:
#RFM Segment Distribution
segment_counts=data['Value Segment'].value_counts().reset_index()
segment_counts.columns=['Value Segment','Count']

In [22]:
segment_counts

,Value Segment,Count
0,Low-Value,435
1,Mid-Value,386
2,High-Value,179


In [23]:
pastel_colors=px.colors.qualitative.Pastel

# Create the bar chart
fig_segment_dist=px.bar(segment_counts,x='Value Segment',y='Count',color='Value Segment',
                        color_discrete_sequence=pastel_colors,title='RFM Value Segment Distribution')
#update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',yaxis_title='Count')

# show the figure
fig_segment_dist.show()


RFM Customer Segments

 'Champions','Potential Loyalists','Cant Lose','Lost'

In [24]:
# create a new column for RFM Customer Segments
data['RFM Customer Segments']=''

# Assign RFM Segments based on the RFM Score
data.loc[data['RMF_Score']>=9,'RFM Customer Segments']='Champions'
data.loc[(data['RMF_Score']>=6)&(data['RMF_Score']<9),'RFM Customer Segments']='Potentail Loyalists'
data.loc[(data['RMF_Score']>=5)&(data['RMF_Score']<6),'RFM Customer Segments']='At Risk Customers'
data.loc[(data['RMF_Score']>=4)&(data['RMF_Score']<5),'RFM Customer Segments']='cant Lose'
data.loc[(data['RMF_Score']>=3)&(data['RMF_Score']<4),'RFM Customer Segments']='Lost'

#print the updated data with RFM Customer Segments
print(data[['CustomerID','RFM Customer Segments']])

     CustomerID RFM Customer Segments
0          8814             cant Lose
1          2188                  Lost
2          4608                  Lost
3          2559                  Lost
4          9482             cant Lose
..          ...                   ...
995        2970   Potentail Loyalists
996        6669   Potentail Loyalists
997        8836   Potentail Loyalists
998        1440   Potentail Loyalists
999        4759   Potentail Loyalists

[1000 rows x 2 columns]


In [25]:
data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RMF_Score,Value Segment,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,302,1,943.31,1,1,2,4,Low-Value,cant Lose
1,2188,2023-04-11,463.70,Product A,176819,London,302,1,463.70,1,1,1,3,Low-Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,302,1,80.28,1,1,1,3,Low-Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,302,1,221.29,1,1,1,3,Low-Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,302,1,739.56,1,1,2,4,Low-Value,cant Lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,242,1,759.62,5,1,2,8,High-Value,Potentail Loyalists
996,6669,2023-06-10,941.50,Product C,987025,New York,242,1,941.50,5,1,2,8,High-Value,Potentail Loyalists
997,8836,2023-06-10,545.36,Product C,512842,London,242,1,545.36,5,1,2,8,High-Value,Potentail Loyalists
998,1440,2023-06-10,729.94,Product B,559753,Paris,242,1,729.94,5,1,2,8,High-Value,Potentail Loyalists


RFM Analysis

distribution of customers accross different RFM Customer segments within each value segment

In [26]:
segment_product_counts=data.groupby(['Value Segment','RFM Customer Segments']).size().reset_index()
segment_product_counts

,Value Segment,RFM Customer Segments,0
0,Low-Value,At Risk Customers,180
1,Low-Value,Champions,0
2,Low-Value,Lost,82
3,Low-Value,Potentail Loyalists,0
4,Low-Value,cant Lose,173
5,Mid-Value,At Risk Customers,0
6,Mid-Value,Champions,0
7,Mid-Value,Lost,0
8,Mid-Value,Potentail Loyalists,386
9,Mid-Value,cant Lose,0


In [27]:
segment_product_counts= data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_product_counts= segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product= px.treemap(segment_product_counts,
                                         path=['Value Segment', 'RFM Customer Segments'],
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

 Distribution Of RFM Values within Champions segment

In [28]:
# Filter the data to include only the customers in the Champions segment
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()

In [29]:
data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RMF_Score,Value Segment,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,302,1,943.31,1,1,2,4,Low-Value,cant Lose
1,2188,2023-04-11,463.70,Product A,176819,London,302,1,463.70,1,1,1,3,Low-Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,302,1,80.28,1,1,1,3,Low-Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,302,1,221.29,1,1,1,3,Low-Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,302,1,739.56,1,1,2,4,Low-Value,cant Lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,242,1,759.62,5,1,2,8,High-Value,Potentail Loyalists
996,6669,2023-06-10,941.50,Product C,987025,New York,242,1,941.50,5,1,2,8,High-Value,Potentail Loyalists
997,8836,2023-06-10,545.36,Product C,512842,London,242,1,545.36,5,1,2,8,High-Value,Potentail Loyalists
998,1440,2023-06-10,729.94,Product B,559753,Paris,242,1,729.94,5,1,2,8,High-Value,Potentail Loyalists


correlation of the recency, frequency, and monetary scores within the champions segment:

In [31]:
correlation_matrix=champions_segment[['RecencyScore','FrequencyScore','MonetaryScore']].corr()
correlation_matrix

,RecencyScore,FrequencyScore,MonetaryScore
RecencyScore,1.000000,-0.571727,-0.474715
FrequencyScore,-0.571727,1.000000,0.390657
MonetaryScore,-0.474715,0.390657,1.000000


In [32]:
# Visualize the correlation matrix using a heatmap
fig_heatmap=go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')
fig_heatmap

 number of customers in all the segments:

In [35]:
segment_counts = data['RFM Customer Segments'].value_counts()
segment_counts

Potentail Loyalists    503
At Risk Customers      180
cant Lose              173
Lost                    82
Champions               62
Name: RFM Customer Segments, dtype: int64

In [33]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig_bar= go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Update the layout
fig_bar.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',yaxis_title='Number of Customers',showlegend=False)
fig_bar.show()


recency, frequency, and monetary scores of all the segments

In [37]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment
rfm_segment_scores = data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()
rfm_segment_scores

<ipython-input-37-e4aaec4c9421>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,RFM Customer Segments,RecencyScore,FrequencyScore,MonetaryScore
0,At Risk Customers,2.344444,1.011111,1.644444
1,Champions,3.806452,3.064516,3.225806
2,Lost,1.000000,1.000000,1.000000
3,Potentail Loyalists,3.918489,1.194831,1.741551
4,cant Lose,1.537572,1.000000,1.462428


In [39]:
# Create a grouped bar chart to compare segment scores
fig_gbar=go.Figure()

#Add bar for Recency score
fig_gbar.add_trace(go.Bar(x=rfm_segment_scores['RFM Customer Segments'],y=rfm_segment_scores['RecencyScore'],
                          name='Recency Score',marker_color='rgb(158,202,225)'))
fig_gbar.add_trace(go.Bar(x=rfm_segment_scores['RFM Customer Segments'],y=rfm_segment_scores['FrequencyScore'],
                          name='Frequency Score',marker_color='rgb(94,158,217)'))
fig_gbar.add_trace(go.Bar(x=rfm_segment_scores['RFM Customer Segments'],y=rfm_segment_scores['MonetaryScore'],
                          name='Monetary Score',marker_color='rgb(32,102,148)'))

# Update the layout
fig_gbar.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig_gbar.show()

# Summary

# RFM Analysis is used to understand and segment customers based on their buying behaviour. RFM stands for recency, frequency, and monetary value, which are three key metrics that provide information about customer engagement, loyalty, and value to a business.